In [ ]:
"""



Author : Elsie Rezinold Yedida





"""

In [ ]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
import tensorflow.keras.layers as tfl


In [ ]:
(train,test),info = tfds.load('horses_or_humans',split=
		['train','test'],
		with_info=True,
		as_supervised=True
		)

print(info)
EPOCHS = 100

In [ ]:
def format_image(image,label):
	image = tf.image.resize(image,(100,100))/255.0
	return image,label

BATCH_SIZE = 32
num_examples = info.splits['train'].num_examples

train_dataset = train.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

test_dataset = test.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)


In [ ]:
image,label = next(iter(train_dataset.take(1)))
def plot(images):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img,ax in zip(images,axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()
plot(image[:5])

In [ ]:
print(label[:5])

In [ ]:

model1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(256, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.8),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model1.compile(loss='BinaryCrossentropy',
              optimizer='adam',
              metrics=['accuracy'])
EPOCHS_MODEL1 = EPOCHS

history1 = model1.fit(
      train_dataset,
      epochs=EPOCHS_MODEL1,
      verbose=1,
      validation_data=test_dataset)

In [ ]:

def identity_block(X, f, filters, initializer=random_uniform):
    F1, F2, F3 = filters
    X_shortcut = X

    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F2, kernel_size = f,strides=(1,1),padding='same',kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F3, kernel_size = 1,strides=(1,1),padding='valid',kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    return X

In [ ]:
def convolutional_block(X, f, filters, s = 2, initializer=glorot_uniform):
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1, 1), padding='same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


In [ ]:
train_dataset1 = train_dataset

test_dataset1 = test_dataset


In [ ]:
def RN(input_shape = (64, 64, 3), classes = 1):
    X_input = Input(input_shape)


    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    # Stage 3
    X = convolutional_block(X,f=3,filters=[128,128,512],s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4
    X = convolutional_block(X,f=3,filters=[256,256,1024],s=2)

    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = convolutional_block(X,f=3,filters=[512,512,2048],s=2)

    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    X = AveragePooling2D()(X)
#    X = MaxPooling2D()(X)


    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)


    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
model2 = RN(input_shape = (100,100, 3), classes = 1)
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
EPOCHS_MODEL2 = EPOCHS
history2 = model2.fit(
      train_dataset1,
      epochs=EPOCHS_MODEL2,
      validation_data=test_dataset1)


In [ ]:


acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']
acc2 = history2.history['accuracy']
val_acc2 = history2.history['val_accuracy']

epochs_range1 = range(EPOCHS_MODEL1)
epochs_range2 = range(EPOCHS_MODEL2)
loss = history1.history['loss']
val_loss = history1.history['val_loss']
loss2 = history2.history['loss']
val_loss2 = history2.history['val_loss']

plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.plot(epochs_range1,acc,label='Training_accuracy for model1')
plt.plot(epochs_range1,val_acc,label='validation_accuracy for model1')
plt.legend(loc='lower right')
plt.title('Training & Validation Accuracy')

plt.subplot(2,1,2)
plt.plot(epochs_range2,acc2,label='Training_accuracy for model2')
plt.plot(epochs_range2,val_acc2,label='validation_accuracy for model2')
plt.legend(loc='lower right')
plt.savefig('./model 1')
plt.show()

plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.plot(epochs_range1,loss,label='loss for model1')
plt.plot(epochs_range1,val_loss,label='validation_loss for model1')
plt.legend(loc='upper right')
plt.title('Training & Validation loss')
plt.subplot(2,1,2)
plt.plot(epochs_range2,loss2,label='loss for model2')
plt.plot(epochs_range2,val_loss2,label='validation_loss for model2')

plt.legend(loc='upper right')

plt.savefig('./model 2')
plt.show()


In [ ]:
def format_image(image,label):
	image = tf.image.resize(image,(160,160))#/255.0
	return image,label

BATCH_SIZE = 32
num_examples = info.splits['train'].num_examples
train_dataset2 = train.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

test_dataset2 = test.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)


In [ ]:
def data_augmenter():
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip('horizontal'))
    data_augmentation.add(RandomRotation(0.2))

    return data_augmentation

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
IMG_SIZE = (160,160)
def horse_model(image_shape=IMG_SIZE, data_augmentation=data_augmenter()):



    input_shape = image_shape + (3,)


    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                   include_top=False,
                                                   weights='imagenet') # From imageNet

    # freeze the base model by making it non trainable
    base_model.trainable = False
    inputs = tf.keras.Input(shape=input_shape)

    # apply data augmentation to the inputs
    x = data_augmentation(inputs)

    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(x)

    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x, training=False)

    # add the new Binary classification layers
    # use global avg pooling to summarize the info in each channel
    x = tfl.GlobalAveragePooling2D()(x)
    # include dropout with probability of 0.2 to avoid overfitting
    x = tfl.Dropout(0.2)(x)

    # use a prediction layer with one neuron (as a binary classifier only needs one)
    outputs = tfl.Dense(1)(x)


    model = tf.keras.Model(inputs, outputs)

    return model

In [ ]:


model3 = horse_model(IMG_SIZE)

In [ ]:
base_learning_rate = 0.001
model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history3 = model3.fit(train_dataset2, epochs=20,batch_size=32,validation_data=test_dataset2,steps_per_epoch=33,validation_steps=8)

In [ ]:


acc = history3.history['accuracy']
val_acc = history3.history['val_accuracy']
epochs_range = range(20)

loss = history3.history['loss']
val_loss = history3.history['val_loss']

plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.plot(epochs_range,acc,label='Training_accuracy for model3')
plt.plot(epochs_range,val_acc,label='validation_accuracy for model3')
plt.legend(loc='lower right')
plt.title('Training & Validation Accuracy')

plt.subplot(2,1,2)
plt.plot(epochs_range,loss,label='loss for model3')
plt.plot(epochs_range,val_loss,label='validation_loss for model3')
plt.legend(loc='upper left')
plt.title('Training & Validation loss')
plt.savefig('./models3')
plt.show()


In [ ]:
model1.evaluate(test_dataset)

In [ ]:
model2.evaluate(test_dataset1)

In [ ]:
model3.evaluate(test_dataset2)